In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
sw = stopwords.words('english')

In [0]:
df = pd.read_csv('./train.csv')

In [66]:
df.head()

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
0,NaN,Australia,Andrew Peace 2007 Peace Family Vineyard Chardo...,"Classic Chardonnay aromas of apple, pear and h...",Peace Family Vineyard,83,10.0,Australia Other,South Eastern Australia,NaN,Andrew Peace,Chardonnay
1,@wawinereport,US,North by Northwest 2014 Red (Columbia Valley (...,This wine is near equal parts Syrah and Merlot...,NaN,89,15.0,Washington,Columbia Valley (WA),Columbia Valley,North by Northwest,Red Blend
2,NaN,Italy,Renato Ratti 2007 Conca (Barolo),Barolo Conca opens with inky dark concentratio...,Conca,94,80.0,Piedmont,Barolo,NaN,Renato Ratti,Nebbiolo
3,@vossroger,France,Domaine l'Ancienne Cure 2010 L'Abbaye White (B...,It's impressive what a small addition of Sauvi...,L'Abbaye,87,22.0,Southwest France,Bergerac Sec,NaN,Domaine l'Ancienne Cure,Bordeaux-style White Blend
4,@vossroger,France,Château du Cèdre 2012 Le Cèdre Vintage Malbec ...,"This ripe, sweet wine is rich and full of drie...",Le Cèdre Vintage,88,33.0,France Other,Vin de Liqueur,NaN,Château du Cèdre,Malbec


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82657 entries, 0 to 82656
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_name           63264 non-null  object 
 1   country             82622 non-null  object 
 2   review_title        82657 non-null  object 
 3   review_description  82657 non-null  object 
 4   designation         59010 non-null  object 
 5   points              82657 non-null  int64  
 6   price               77088 non-null  float64
 7   province            82622 non-null  object 
 8   region_1            69903 non-null  object 
 9   region_2            35949 non-null  object 
 10  winery              82657 non-null  object 
 11  variety             82657 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 7.6+ MB


In [68]:
df.dropna(axis=0)

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery,variety
6,@wawinereport,US,Gordon Estate 2012 Tradition Red (Columbia Val...,"Half Syrah and half Cabernet Sauvignon, with 4...",Tradition,91,45.0,Washington,Columbia Valley (WA),Columbia Valley,Gordon Estate,Red Blend
7,@vboone,US,Rock Wall 2014 Monte Rosso Reserve Zinfandel (...,"High-toned red fruit stars in this wine, a nic...",Monte Rosso Reserve,90,50.0,California,Sonoma County,Sonoma,Rock Wall,Zinfandel
8,@paulgwine,US,Elizabeth Chambers 2014 Falcon Glen Vineyard P...,"Displaying typical McMinnville AVA subtlety, t...",Falcon Glen Vineyard,91,48.0,Oregon,McMinnville,Willamette Valley,Elizabeth Chambers,Pinot Noir
10,@vboone,US,Cuvaison 2014 Estate Grown Sauvignon Blanc (Ca...,With an alluring bouquet of fresh-cut jasmine ...,Estate Grown,90,18.0,California,Carneros,Napa-Sonoma,Cuvaison,Sauvignon Blanc
34,@paulgwine,US,Durant 2013 Raven Chardonnay (Dundee Hills),"Peach pit, apple and pear fruit flavors combin...",Raven,92,25.0,Oregon,Dundee Hills,Willamette Valley,Durant,Chardonnay
...,...,...,...,...,...,...,...,...,...,...,...,...
82630,@vboone,US,Yorkville Cellars 2010 Randle Hill Vineyard Sa...,"A glass of pretty aromatics, grapefruit and or...",Randle Hill Vineyard,88,18.0,California,Yorkville Highlands,North Coast,Yorkville Cellars,Sauvignon Blanc
82632,@vboone,US,un4seen 2009 Red blend Red (California),"Dark like a cave in color, this big red blend ...",Red blend,85,10.0,California,California,California Other,un4seen,Red Blend
82639,@paulgwine,US,Archery Summit 2012 Red Hills Estate Pinot Noi...,"Showing a light whiff of funk, this seems a bi...",Red Hills Estate,92,85.0,Oregon,Dundee Hills,Willamette Valley,Archery Summit,Pinot Noir
82641,@gordone_cellars,US,The Withers Winery 2013 Mr. Burgess Red (El Do...,"Imposing, concentrated and tannic, this Syrah-...",Mr. Burgess,92,36.0,California,El Dorado,Sierra Foothills,The Withers Winery,Rhône-style Red Blend


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82657 entries, 0 to 82656
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_name           63264 non-null  object 
 1   country             82622 non-null  object 
 2   review_title        82657 non-null  object 
 3   review_description  82657 non-null  object 
 4   designation         59010 non-null  object 
 5   points              82657 non-null  int64  
 6   price               77088 non-null  float64
 7   province            82622 non-null  object 
 8   region_1            69903 non-null  object 
 9   region_2            35949 non-null  object 
 10  winery              82657 non-null  object 
 11  variety             82657 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 7.6+ MB


In [0]:
x = df['review_description']
y = df['variety']

In [71]:
len(set(y))

28

In [72]:
labels = LabelEncoder()
y = labels.fit_transform(y)
y

array([ 5, 17, 11, ...,  6,  0,  3])

In [0]:
x = x.str.lower()

In [74]:
x

0        classic chardonnay aromas of apple, pear and h...
1        this wine is near equal parts syrah and merlot...
2        barolo conca opens with inky dark concentratio...
3        it's impressive what a small addition of sauvi...
4        this ripe, sweet wine is rich and full of drie...
                               ...                        
82652    hot earth, baked cherry and mild vanilla aroma...
82653    pungent, high-pitched notes of turmeric and dr...
82654    this is a wine that's not just lively and frui...
82655    an impressive wine, it already shows signs of ...
82656    a big, rich, vital cabernet with immediate att...
Name: review_description, Length: 82657, dtype: object

In [0]:
without_sw = []
for word_i in x:
    without_sw.append(' '.join([re.sub('[0-9\W_]', '', word) for word in word_i.split() if not word in sw]))
x = without_sw

In [76]:
x

['classic chardonnay aromas apple pear hay lead palate marked decent intensity also bit sweetness orange candy notes run rather short finish',
 'wine near equal parts syrah merlot balance cabernet sauvignon aromas blue fruit vanilla cherry herb lead fullbodied pitfruit flavors bring sense deliciousness hard resist',
 'barolo conca opens inky dark concentration soothing aromas black fruit spice cola plum prune dried lavender buds nose presents ethereal delicate aromas mouth delivers thicker flavors chocolate mocha drink ',
 'impressive small addition sauvignon gris muscadelle sauvignonsémillon blendit turns exotic wine spice lychee flavors',
 'ripe sweet wine rich full dried fresh fruit flavors spicy touch spirit palate well bold blackplum fruit dense tannins sweet aftertaste drink ',
 'strident aromas licorice herbs berries mint make interesting palate light fresh raspberry red currant flavors theres lot heft extract depth wine likable unhindered excess oak such enjoyable due unchallen

In [0]:
countVectorizer = CountVectorizer()
x = countVectorizer.fit_transform(x)

In [78]:
x[0]

<1x35458 sparse matrix of type '<class 'numpy.int64'>'
	with 21 stored elements in Compressed Sparse Row format>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [0]:
from keras.optimizers import *

In [82]:
len(countVectorizer.get_feature_names())

35458

In [83]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=len(countVectorizer.get_feature_names())))
model.add(Dense(100, activation='relu'))
model.add(Dense(units=y.max()+1, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, verbose=1, validation_data=(X_test, y_test))

Train on 66125 samples, validate on 16532 samples
Epoch 1/1
66125/66125 [==============================] - 17s 254us/step - loss: 1.2363 - accuracy: 0.6246 - val_loss: 0.9581 - val_accuracy: 0.7010


In [84]:
scores = model.evaluate(X_test, y_test, verbose=1)
print ('Accuracy', scores[1])

16532/16532 [==============================] - 2s 147us/step
Accuracy 0.7010040879249573


In [85]:
model.metrics_names

['loss', 'accuracy']

In [0]:
dfT = pd.read_csv('./test.csv')

In [87]:
dfT.dropna(axis=0)

,user_name,country,review_title,review_description,designation,points,price,province,region_1,region_2,winery
0,@paulgwine,US,Boedecker Cellars 2011 Athena Pinot Noir (Will...,Nicely differentiated from the companion Stewa...,Athena,88,35.0,Oregon,Willamette Valley,Willamette Valley,Boedecker Cellars
14,@paulgwine,US,Brittan Vineyards 2014 Gestalt Block Pinot Noi...,This black-fruited offering has the winery's s...,Gestalt Block,93,60.0,Oregon,McMinnville,Willamette Valley,Brittan Vineyards
16,@vboone,US,Mounts 2014 Estate Grown Cabernet Sauvignon (D...,"This is a seductive wine, concentrated and ful...",Estate Grown,88,45.0,California,Dry Creek Valley,Sonoma,Mounts
18,@paulgwine,US,Novelty Hill 2009 Stillwater Creek Vineyard Ca...,Novelty Hill's estate-grown Cabernet Sauvignon...,Stillwater Creek Vineyard,89,30.0,Washington,Columbia Valley (WA),Columbia Valley,Novelty Hill
26,@vboone,US,Medlock Ames 2013 Heritage Merlot (Alexander V...,"Made with 21% Cabernet Sauvignon, 4% Malbec an...",Heritage,87,50.0,California,Alexander Valley,Sonoma,Medlock Ames
...,...,...,...,...,...,...,...,...,...,...,...
20653,@wawinereport,US,Almquist 2010 Garagiste Rosso Red (Washington),This wine is a blend of Sangiovese (65%) and B...,Garagiste Rosso,90,20.0,Washington,Washington,Washington Other,Almquist
20654,@wawinereport,US,J. Bookwalter 2015 Suspense Conner Lee Vineyar...,"Aromas of herb, freshly sliced green pepper, c...",Suspense Conner Lee Vineyard,90,60.0,Washington,Columbia Valley (WA),Columbia Valley,J. Bookwalter
20659,@mattkettmann,US,Blackjack 2012 Black Cap Reserve Chardonnay (S...,The inviting scents of melting butter on hot d...,Black Cap Reserve,89,48.0,California,Santa Barbara County,Central Coast,Blackjack
20660,@gordone_cellars,US,Yorkville Cellars 2013 Rennie Vineyard Caberne...,"Clearly focused and fruit-driven, this wine ha...",Rennie Vineyard,91,34.0,California,Yorkville Highlands,North Coast,Yorkville Cellars


In [0]:
xT = dfT['review_description']

In [0]:
xT = xT.str.lower()

In [0]:
without_sw = []
for word_i in xT:
    without_sw.append(' '.join([re.sub('[0-9\W_]', '', word) for word in word_i.split() if not word in sw]))
xT = without_sw

In [0]:
# countVectorizerT = CountVectorizer()
xT = countVectorizer.transform(xT)

In [92]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               3545900   
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 28)                2828      
Total params: 3,558,828
Trainable params: 3,558,828
Non-trainable params: 0
_________________________________________________________________


In [0]:
yT = model.predict(xT)

In [95]:
yT[0]

array([1.4197144e-05, 5.8732624e-08, 2.4150619e-04, 6.6075538e-04,
       1.6824012e-04, 2.1737812e-04, 9.2814771e-06, 9.4565430e-06,
       4.7432757e-05, 4.6556328e-05, 2.7423678e-04, 1.2869161e-04,
       4.6277601e-06, 9.0818154e-05, 9.7917087e-02, 3.7787458e-06,
       4.1031205e-07, 1.4173203e-03, 2.6138514e-04, 2.2241558e-05,
       3.4948526e-04, 8.5780129e-04, 9.0140138e-06, 2.9419322e-04,
       2.2460332e-03, 8.3336444e-04, 3.4171502e-05, 2.9229186e-03],
      dtype=float32)

In [96]:
np.argmax(yT[0])

14

In [97]:
len(yT)

20665

In [99]:
len(xT.toarray())

20665

In [0]:
yT_req = [np.argmax(yT[i]) for i in range(len(yT))]

In [102]:
yT_req

[14,
 9,
 5,
 9,
 21,
 26,
 0,
 3,
 17,
 23,
 10,
 4,
 20,
 14,
 9,
 19,
 3,
 14,
 3,
 5,
 10,
 14,
 25,
 15,
 5,
 20,
 3,
 5,
 22,
 17,
 19,
 3,
 24,
 3,
 15,
 12,
 21,
 5,
 3,
 5,
 3,
 3,
 18,
 25,
 8,
 3,
 27,
 19,
 15,
 27,
 14,
 19,
 5,
 14,
 22,
 5,
 15,
 26,
 22,
 5,
 23,
 22,
 0,
 2,
 14,
 3,
 8,
 20,
 2,
 14,
 0,
 13,
 23,
 11,
 3,
 10,
 5,
 21,
 3,
 5,
 14,
 27,
 22,
 17,
 5,
 0,
 4,
 14,
 27,
 17,
 14,
 19,
 20,
 24,
 24,
 0,
 18,
 23,
 3,
 14,
 3,
 14,
 0,
 22,
 21,
 17,
 15,
 3,
 11,
 17,
 3,
 25,
 5,
 20,
 19,
 23,
 14,
 5,
 22,
 20,
 3,
 17,
 16,
 5,
 5,
 5,
 13,
 25,
 15,
 14,
 15,
 14,
 9,
 14,
 3,
 25,
 22,
 20,
 22,
 0,
 0,
 24,
 23,
 3,
 24,
 26,
 15,
 14,
 27,
 0,
 5,
 23,
 14,
 3,
 11,
 20,
 5,
 20,
 8,
 3,
 3,
 17,
 17,
 14,
 9,
 26,
 0,
 5,
 0,
 5,
 5,
 21,
 14,
 19,
 22,
 4,
 17,
 20,
 21,
 10,
 19,
 8,
 13,
 14,
 0,
 26,
 5,
 3,
 17,
 0,
 5,
 19,
 14,
 24,
 24,
 27,
 14,
 23,
 21,
 3,
 14,
 5,
 14,
 3,
 3,
 25,
 14,
 5,
 24,
 5,
 5,
 8,
 3,
 14,
 7,
 24,
 5,
 

In [0]:
df_variety = pd.DataFrame({'variety':yT_req})

In [104]:
df_variety.head()

,variety
0,14
1,9
2,5
3,9
4,21


In [0]:
df_variety.to_csv('predicted_variety.csv', index=False)

In [0]:
df_features = pd.DataFrame({'features':countVectorizer.get_feature_names()})

In [109]:
df_features.head()

,features
0,aand
1,aaron
2,aas
3,ab
4,abacela


In [0]:
df_features.to_csv('features.csv', index=False)